In [6]:
!pip3 install selenium

In [1]:
!pip install ipywidgets

     -------------------------------------- 123.4/123.4 KB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 2.9 MB/s eta 0:00:00
     -------------------------------------- 245.1/245.1 KB 3.8 MB/s eta 0:00:00


In [7]:
!pip3 install beautifulsoup4

In [10]:
!pip3 install lxml

     ---------------------------------------- 3.6/3.6 MB 5.2 MB/s eta 0:00:00


In [48]:
!pip3 install six

In [463]:
import requests, time, random
from bs4 import BeautifulSoup
from selenium import webdriver
from six import iteritems

In [789]:
browser = webdriver.Chrome('driver/chromedriver.exe')

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [790]:
browser.get('https://www.linkedin.com/uas/login')
file = open('config.txt')
lines = file.readlines()
username = lines[0]
password = lines[1]


elementID = browser.find_element_by_id('username')
elementID.send_keys(username)

elementID = browser.find_element_by_id('password')
elementID.send_keys(password)

elementID.submit()

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  # This is added back by InteractiveShellApp.init_path()


In [775]:
# https://www.linkedin.com/check/add-phone?country_code=in
# print(browser)
# elementID = browser.find_element_by_id('username')
# print('elementID', elementID)

In [776]:
# link = 'https://www.linkedin.com/in/nikhil-gujrati-1408931ab/'
link = 'https://www.linkedin.com/in/anupam-agrawal-a822b011/'
# link = 'https://www.linkedin.com/in/rishabh-singh-61b706114/'
browser.get(link)

In [777]:
SCROLL_PAUSE_TIME = 5

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

for i in range(3):
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [778]:
def printDict(_dict):
    for key, value in iteritems(_dict):
        print(key,':',value)

def printListDic(all_exp):
    for _dict in all_exp:
        printDict(_dict)
        print()

In [779]:
import re

In [780]:
src = browser.page_source
soup = BeautifulSoup(src, 'lxml')

In [781]:
def getProfileName(bs):
    name_div = bs.find('div', {'class': 'mt2 relative'})
    name_loc = name_div.find_all('div')
    name = name_loc[0].find('h1').get_text().strip()
    return str(name)

def getProfileLocation(bs):
    name_div = bs.find('div', {'class': 'mt2 relative'})
    name_loc = name_div.find_all('div')
    if len(name_loc) == 0:
        return 'None'
    loc = name_loc[-1].find('span').get_text().strip()
    return str(loc)

def getBasicProfileInfo(bs):
    info = {}
    info['Name'] = getProfileName(bs)
    info['Location'] = getProfileLocation(bs)
    return info
basicProfileInfo = getBasicProfileInfo(soup)
printDict(basicProfileInfo)

Name : Anupam Agrawal
Location : Allahabad, Uttar Pradesh, India


In [786]:
def getExpURLID(bs):
    ids = []
    href_classes = bs.findAll('a', {'data-field': 'experience_company_logo'})
    for href_class in href_classes:
        if href_class is None:
            continue
        exp_url = href_class.attrs.get('href', None)
        if exp_url is None:
            continue
        ids.append(exp_url)
    return ids

def getExpTitle(all_exp_spans):
    if len(all_exp_spans) < 2:
        return 'None'
    job_title = all_exp_spans[1].get_text().strip()
    return str(job_title)

def getExpCompany(all_exp_spans):
    if len(all_exp_spans) < 4:
        return 'None'
    double_text = str(all_exp_spans[3].get_text().strip())
    strip_index = len(double_text)//2
    company = double_text[0:strip_index].split(' · ')[0]
    return str(company)

def getExpTimePeriod(all_exp_spans):
    if len(all_exp_spans) < 7:
        return 'None'
    time_period = all_exp_spans[6].get_text().strip().split(' · ')[0]
    return str(time_period)

def getExpInfo(exp):
    info = {}
    all_exp_spans = exp.find_all('span')
    job_title = getExpTitle(all_exp_spans)
    company = getExpCompany(all_exp_spans)
    time_period = getExpTimePeriod(all_exp_spans)
    
    info['Job Title'] = job_title
    info['Company'] = company
    info['Time Period'] = time_period
    return info
    
def getExperienceSetions(bs):
    profile_sections = bs.findAll('section')
    exp_profile_section = None
    for profile_section in profile_sections:
        x = profile_section.find('div', {'id': 'experience'})
        if x is not None:
            exp_profile_section = profile_section
            break
    if exp_profile_section is None:
        return {}
    exp_sections = exp_profile_section.find_all('div', {'class': 'display-flex flex-column full-width align-self-center'})
    return exp_sections

def getAllExperieceInfo(bs):
    all_experiences = []
    company_ids = getExpURLID(bs)
    exp_sections = getExperienceSetions(bs)
    for exp in exp_sections:
        all_experiences.append(getExpInfo(exp))
    for i in range(len(all_experiences)):
        if i >= len(company_ids):
            all_experiences[i]['URL ID'] = 'None'
            continue
        all_experiences[i]['URL ID'] = str(company_ids[i])
    return all_experiences

all_experiences = getAllExperieceInfo(soup)
printListDic(all_experiences)

Job Title : Professor
Company : Indian Institute Of Information Technology
Time Period : Nov 2000 - Present
URL ID : https://www.linkedin.com/company/497317/

Job Title : Senior Scientist
Company : DRDO, Ministry of Defence, Govt. of India
Time Period : Jan 1989 - Oct 2000
URL ID : https://www.linkedin.com/company/30205475/



In [783]:
def getEduSchoolID(edu):
    href_class = edu.findAll('a')
    if href_class is None or len(href_class) == 0:
        return 'None'
    school_url = href_class[0].attrs.get('href', None)
    if school_url is None:
        return 'None'
    return str(school_url)

def getEduSchool(all_edu_spans):
    if len(all_edu_spans) < 2:
        return 'None'
    job_title = all_edu_spans[1].get_text().strip()
    return str(job_title)

def getEduCourse(all_edu_spans):
    if len(all_edu_spans) < 4:
        return 'None'
    course_and_discipline = all_edu_spans[4].get_text().strip().split(',')
    if len(course_and_discipline) < 1:
        return 'None'
    course = course_and_discipline[0]
    return str(course)

def getEduDiscipline(all_edu_spans):
    if len(all_edu_spans) < 4:
        return 'None'
    line_text = all_edu_spans[4].get_text()
    course_and_discipline = line_text.strip().split(',')
    if len(course_and_discipline) < 2:
        return 'None'
    ind = line_text.find(', ')
    discipline = line_text[ind+1:]
    return str(discipline)

def getEduTimePeriod(all_edu_spans):
    if len(all_edu_spans) < 8:
        return 'None'
    time_period = all_edu_spans[7].get_text().strip()
    return str(time_period)

def getEduInfo(edu):
    info = {}
    # print(edu)
    all_edu_spans = edu.find_all('span')
    school_url_id = getEduSchoolID(edu)
    school = getEduSchool(all_edu_spans)
    course = getEduCourse(all_edu_spans)
    discipline = getEduDiscipline(all_edu_spans)
    time_period = getEduTimePeriod(all_edu_spans)
    info['URL ID'] = school_url_id
    info['Insititution'] = school
    info['Course'] = course
    info['Discipline'] = discipline
    info['Time Period'] = time_period
    return info
    
def getEducationSetions(bs):
    profile_sections = bs.findAll('section')
    edu_profile_section = None
    for profile_section in profile_sections:
        x = profile_section.find('div', {'id': 'education'})
        if x is not None:
            edu_profile_section = profile_section
            break
    if edu_profile_section is None:
        return {}
    edu_sections = edu_profile_section.find_all('div', {'class': 'display-flex flex-column full-width align-self-center'})
    return edu_sections

def getAllEducationInfo(bs):
    all_educations = []
    edu_sections = getEducationSetions(bs)
    for edu in edu_sections:
        all_educations.append(getEduInfo(edu))
    return all_educations

all_educations = getAllEducationInfo(soup)
printListDic(all_educations)

URL ID : https://www.linkedin.com/search/results/all/?keywords=University+of+Bedfordshire%2C+United+Kingdom
Insititution : University of Bedfordshire, United Kingdom
Course : Post-Doctoral Research Fellow
Discipline :  Computer Science
Time Period : 2007 - 2009

URL ID : https://www.linkedin.com/company/497317/
Insititution : Indian Institute Of Information Technology
Course : PhD
Discipline :  Computer Science
Time Period : 2001 - 2006

URL ID : https://www.linkedin.com/search/results/all/?keywords=Indian+Institute+of+Technology+Madras
Insititution : Indian Institute of Technology Madras
Course : Master of Technology - MTech
Discipline :  Computer Science
Time Period : 1993 - 1995



In [468]:
def is_from_IIITA(all_educations):
    IIITA_ID = 'https://www.linkedin.com/company/497317/'
    for each_education in all_educations:
        if each_education['Insititution ID'] == IIITA_ID:
            return True
    return False
print('Is IIIT-A student?', is_from_IIITA(all_educations))

Is IIIT-A student? False


In [461]:
profiles = ['https://www.linkedin.com/in/shivam-gupta-52302516b/',
           'https://www.linkedin.com/in/isha-goel30/',
           'https://www.linkedin.com/in/richik-bhattacharjee/',
           'https://www.linkedin.com/in/sougata-das-4132a616b/',
           'https://www.linkedin.com/in/abhi4gupta/',]

# for profile_url in profiles:
#     browser.get(profile_url)
#     print(isConnection(profile_url), profile_url)
#     break
# profile_url = profiles[1]
# browser.get(profile_url)

In [578]:
def isConnection(profile_url):
    # pvs-profile-actions 
    src = browser.page_source
    bs = BeautifulSoup(src, 'lxml')
    actions_div = bs.find('div', {'class': 'pvs-profile-actions'})
    # print(actions_div)
    actions = actions_div.findAll('div', {'class': 'entry-point'})
    if actions is None or len(actions) == 0:
        return False
    if 'aria-hidden=\"true\"' in str(actions):
        return False
    return True
for profile_url in profiles:
    browser.get(profile_url)
    time.sleep(2)
    print(isConnection(profile_url), profile_url)

True https://www.linkedin.com/in/shivam-gupta-52302516b/
False https://www.linkedin.com/in/isha-goel30/
True https://www.linkedin.com/in/richik-bhattacharjee/
True https://www.linkedin.com/in/sougata-das-4132a616b/
True https://www.linkedin.com/in/abhi4gupta/


In [562]:
def connectionStatus(profile_url):
    browser.get(profile_url)
    time.sleep(2)
    src = browser.page_source
    bs = BeautifulSoup(src, 'lxml')
    actions_div = bs.find('div', {'class': 'pvs-profile-actions'})
    # print(str(actions_div))
    if actions_div is None:
        return "Error"
    actions = actions_div.findAll('div', {'class': 'entry-point'})
    if ' to connect' in str(actions_div):
        return "Connect"
    if 'Pending' in str(actions_div):
        return "Pending"
    if actions is None or len(actions) == 0:
        return "Unknown"
    return "Connected"
profiles2 = [
            'https://www.linkedin.com/in/siddhi-singh-8653801b1/',
            'https://www.linkedin.com/in/richik-bhattacharjee/',
            'https://www.linkedin.com/in/isha-goel30/',
            'https://www.linkedin.com/in/sougata-das-4132a616b/',
           'https://www.linkedin.com/in/abhi4gupta/',
            ]
for profile_url in profiles2:
    print(connectionStatus(profile_url), profile_url)
    # break

Connect https://www.linkedin.com/in/siddhi-singh-8653801b1/
Connected https://www.linkedin.com/in/richik-bhattacharjee/
Pending https://www.linkedin.com/in/isha-goel30/
Connected https://www.linkedin.com/in/sougata-das-4132a616b/
Connected https://www.linkedin.com/in/abhi4gupta/


In [563]:
visitingProfileID = '/in/aarya-tadvalkar-092650193/'
fullLink = 'https://www.linkedin.com' + visitingProfileID
browser.get(fullLink)

In [566]:
visitedProfiles = []
profilesQueued = []

def getPeopleAlsoViewedSection(bs):
    profile_sections = bs.findAll('section')
    req_profile_section = None
    for profile_section in profile_sections:
        if 'People also viewed' in str(profile_section):
            return profile_section
    return None

def getNewProfileIDs(soup, profilesQueued):
    profilesID = []
    # return soup
    people_also_viewed_section = getPeopleAlsoViewedSection(soup)
    # print(people_also_viewed_section)
    all_links = people_also_viewed_section.findAll('a')
    for link in all_links:
        userID = link.get('href')
        if((userID is not None) and (userID not in profilesQueued) and (userID not in visitedProfiles)):
            profilesID.append(userID)
    return profilesID

getNewProfileIDs(BeautifulSoup(browser.page_source), profilesQueued)

['/in/mrudula-ingale/',
 '/in/vaishnavi-patil-93579816b/',
 '/in/midhat-shaikh-2001/',
 '/in/mihir2510/',
 '/in/devanshi-shah-09/']

In [567]:
LINKEDIN_BASE_URL = 'https://www.linkedin.com'

In [632]:
def getFollowersList(bs):
    followers_url = 'https://www.linkedin.com/feed/followers/'
    browser.get(followers_url)
    time.sleep(2)
    src = browser.page_source
    bs = BeautifulSoup(src, 'lxml')
    lis = []
    all_followers = bs.find('div', {'class': 'feed-following'})
    if all_followers is None:
        return lis
    for each_follower in all_followers.findAll('a'):
        follower_id = each_follower.get('href')
        if follower_id is None:
            continue
        lis.append(follower_id)
    return list(set(list(lis)))
followers = getFollowersList(bs)
print(followers)

['/in/abhi4gupta/', '/in/abhirup-das-740981178/', '/in/shivam-gupta-52302516b/', '/in/richik-bhattacharjee/', '/in/utkarsh-pandey-95015b170/', '/in/sougata-das-4132a616b/', '/in/subharoop-ghosh-943669150/', '/in/aniket-mallick-75572416b/', '/in/isha-goel30/']


In [570]:
# import mysql.connector

# mydb = mysql.connector.connect(
#   host="localhost",
#   user="root",
#   password="pass"
# )

# print(mydb)

def getIIITFollowers(followers):
    IIITFollowers = []
    for follower in followers:
        follower_url = LINKEDIN_BASE_URL + follower
        browser.get(follower_url)
        time.sleep(2)
        src = browser.page_source
        bs = BeautifulSoup(src, 'lxml')
        # basicProfileInfo = getBasicProfileInfo(bs)
        # all_experiences = getAllExperieceInfo(bs)
        all_educations = getAllEducationInfo(bs)
        if is_from_IIITA(all_educations):
            IIITFollowers.append(follower)
        
    return IIITFollowers
IIITFollowers = getIIITFollowers(followers)

In [571]:
print(IIITFollowers)

['/in/abhi4gupta/']


In [793]:
def send_connection_request_with_text(profile_url, text):
    if connectionStatus(profile_url) == 'Connect':
        src = browser.page_source
        bs = BeautifulSoup(src, 'lxml')
        
        actions_divs = bs.findAll('div', {'class': 'pvs-profile-actions'})
        if len(actions_divs) == 0:
            return
        ids = []
        for actions_div in actions_divs:
            buttons = actions_div.findAll('button')
            if len(buttons) == 0:
                continue
            _id = None
            for button in buttons:
                cnnt = button.get('aria-label')
                if cnnt is None:
                    continue
                if ' to connect' in str(cnnt):
                    _id = button.get('id')
                    break
            if _id is None:
                continue
            ids.append(_id)
        if len(ids) == 0:
            return
        browser.find_element_by_id(ids[-1]).click()
        src = browser.page_source
        bs = BeautifulSoup(src, 'lxml')
        _id = None
        for x in bs.find_all('button'):
            if x.get('aria-label') == 'Add a note':
                _id = x.get('id')
                break
        if _id is None:
            return
        browser.find_element_by_id(_id).click()
        INV_REQUEST = 'Please accept my request.'
        elementID = browser.find_element_by_id('custom-message')
        elementID.send_keys(INV_REQUEST)
        
        src = browser.page_source
        bs = BeautifulSoup(src, 'lxml')
        _id = None
        for x in bs.find_all('button'):
            if x.get('aria-label') == 'Send now':
                _id = x.get('id')
                break
        if _id is None:
            return
        browser.find_element_by_id(_id).click()
        
send_connection_request_with_text('https://www.linkedin.com/in/kapil-tomar-a2b31118b/', 'Hi! Please accept my connection request')

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:37: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:51: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


In [696]:
# BeautifulSoup(browser.page_source)

In [787]:
def send_text_to_connection(profile_url, text):
    # print(connectionStatus(profile_url))
    if connectionStatus(profile_url) == 'Connected':
        src = browser.page_source
        bs = BeautifulSoup(src, 'lxml')
        x = bs.find('a', {'class': 'message-anywhere-button pvs-profile-actions__action artdeco-button'})
        if x is None:
            return
        link = x.get('href')
        if link is None:
            return
        full_link = 'https://www.linkedin.com' + link
        browser.get(full_link)
        
        src = browser.page_source
        bs = BeautifulSoup(src, 'lxml')
        
        message_text = browser.find_element_by_css_selector('div.msg-form__contenteditable > p')
        message_text.send_keys(text)
        message_text.submit()
        
send_text_to_connection('https://www.linkedin.com/in/abhi4gupta/', 'Dummy Text by bot')

WebDriverException: Message: chrome not reachable
  (Session info: chrome=100.0.4896.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x00779943+2595139]
	Ordinal0 [0x0070C9F1+2148849]
	Ordinal0 [0x006043F0+1065968]
	Ordinal0 [0x005F87C2+1017794]
	Ordinal0 [0x005F8FF8+1019896]
	Ordinal0 [0x005FA892+1026194]
	Ordinal0 [0x005F4219+999961]
	Ordinal0 [0x00605860+1071200]
	Ordinal0 [0x0065B2D2+1422034]
	Ordinal0 [0x0064B806+1357830]
	Ordinal0 [0x00626086+1204358]
	Ordinal0 [0x00626F96+1208214]
	GetHandleVerifier [0x0091B232+1658114]
	GetHandleVerifier [0x009D312C+2411516]
	GetHandleVerifier [0x0080F261+560433]
	GetHandleVerifier [0x0080E366+556598]
	Ordinal0 [0x0071286B+2173035]
	Ordinal0 [0x007175F8+2192888]
	Ordinal0 [0x007176E5+2193125]
	Ordinal0 [0x007211FC+2232828]
	BaseThreadInitThunk [0x7571FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77097A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77097A6E+238]


In [757]:
import InfoScraper
   
profile_info = InfoScraper.InfoScraper('https://www.linkedin.com/in/abhi4gupta/')
# print(profile_info.all_experiences)
print(profile_info)

NameError: name 'self' is not defined